# Georgia ESOL in Public Schools - EDA

In [1]:
# import our favorite libraries
import pandas as pd
import requests

## I. Scrape the data from the Georgia DOE

In [55]:
def file_reader(url):
    if url[-4:] == 'xlsx':
        return pd.read_excel(url)
    else:
        return pd.read_csv(url)

In [57]:
# set aside the customized urls for the CSV files that we're looking to read in to pandas DFs
data_urls = ['https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2012.xlsx',
            'https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2013.csv',
            'https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2014_Jan_15th_2015.csv',
            'https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2015_DEC_1st_2016.csv',
            'https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2016_DEC_1st_2016.csv',
            'https://gosa.georgia.gov/sites/gosa.georgia.gov/files/related_files/site_page/Enrollment_by_Subgroups_Programs_2017_DEC_1st_2017.csv',
            'https://download.gosa.ga.gov/2018/Enrollment_by_Subgroups_Programs_2018_DEC_10th_2018.csv'
             ]

In [62]:
# scrape the CSVs into pandas DFs, one year per
dfs = list(map(file_reader, data_urls))

In [71]:
# we see that the most recent DF has two extra columns at the end, which we'll remove for now to allow for concatenation
# these features are % male and % female, which aren't relevant for us here
for df in dfs:
    print(len(df.columns))
    
dfs[-1].drop(list(dfs[-1].columns[-2:]), axis=1, inplace=True)

35
35
35
35
35
35
37


In [72]:
# concatenate all years' data into one master dataframe
esl_df = pd.concat(dfs, axis=0)

In [75]:
# sneak-peek of our DF
esl_df.head(15)

,DETAIL_LVL_DESC,INSTN_NUMBER,SCHOOL_DSTRCT_CD,LONG_SCHOOL_YEAR,INSTN_NAME,SCHOOL_DSTRCT_NM,GRADES_SERVED_DESC,ENROLL_PERCENT_ASIAN,ENROLL_PERCENT_NATIVE,ENROLL_PERCENT_BLACK,...,ENROLL_COUNT_ESOL,ENROLL_PCT_ESOL,ENROLL_COUNT_SPECIAL_ED_PK,ENROLL_PCT_SPECIAL_ED_PK,ENROLL_COUNT_VOCATION_9_12,ENROLL_PCT_VOCATION_9_12,ENROLL_COUNT_ALT_PROGRAMS,ENROLL_PCT_ALT_PROGRAMS,ENROLL_COUNT_GIFTED,ENROLL_PCT_GIFTED
0,School,0103,601,2011-12,Appling County High School,Appling County,"09,10,11,12",1.0,0.0,22.0,...,8.0,0.8,0.0,0.0,580.0,60.5,45.0,4.7,76.0,7.9
1,School,0109,601,2011-12,Baxley Wilderness Institute,Appling County,"07,08,09,10",0.0,0.0,64.0,...,0.0,0.0,0.0,0.0,4.0,30.8,0.0,0.0,NaN,NaN
2,School,0177,601,2011-12,Appling County Elementary School,Appling County,"03,04,05",1.0,0.0,31.0,...,9.0,1.7,0.0,0.0,NaN,NaN,0.0,0.0,22.0,4.1
3,School,0195,601,2011-12,Appling County Middle School,Appling County,"06,07,08",1.0,0.0,21.0,...,15.0,2.0,0.0,0.0,NaN,NaN,12.0,1.6,68.0,9.0
4,School,0277,601,2011-12,Appling County Primary School,Appling County,"PK,KK,01,02",1.0,0.0,28.0,...,82.0,13.6,31.0,5.1,NaN,NaN,0.0,0.0,NaN,NaN
5,School,1050,601,2011-12,Altamaha Elementary School,Appling County,"PK,KK,01,02,03,04,05",0.0,0.0,3.0,...,9.0,2.7,0.0,0.0,NaN,NaN,0.0,0.0,12.0,3.6
6,School,5050,601,2011-12,Fourth District Elementary School,Appling County,"PK,KK,01,02,03,04,05",0.0,0.0,12.0,...,6.0,5.0,1.0,0.8,NaN,NaN,0.0,0.0,10.0,8.3
7,District,ALL,601,2011-12,All Column Values,Appling County,"PK,KK,01,02,03,04,05,06,07,08,09,10,11,12",1.0,0.0,22.0,...,129.0,3.9,32.0,1.0,584.0,60.1,57.0,1.7,188.0,5.6
8,School,0103,602,2011-12,Atkinson County High School,Atkinson County,"09,10,11,12",0.0,0.0,18.0,...,10.0,2.4,0.0,0.0,300.0,70.9,9.0,2.1,50.0,11.8
9,School,0111,602,2011-12,Atkinson County Middle School,Atkinson County,"06,07,08",0.0,0.0,20.0,...,11.0,2.9,0.0,0.0,NaN,NaN,6.0,1.6,24.0,6.3


### Inital thoughts
The data is fairly simple. For each year from 2011-2018, each public school in Georgia has a row that contains ~30 data points regarding its demographic makeup. The only features we care about for this analysis are those relating to ESOL students. We will clean up our data a bit and then venture into EDA.

## II. Clean up the Data

In [76]:
# drop pre-aggregated columns
esl_df = esl_df.loc[esl_df.DETAIL_LVL_DESC.str.lower() != 'district']

In [77]:
# map columns to all lowercase
esl_df.columns = map(str.lower, esl_df.columns)

In [80]:
esl_df.describe()

,enroll_percent_asian,enroll_percent_native,enroll_percent_black,enroll_percent_hispanic,enroll_percent_multiracial,enroll_percent_white,enroll_percent_migrant,enroll_percent_ed,enroll_percent_swd,enroll_percent_lep,...,enroll_count_esol,enroll_pct_esol,enroll_count_special_ed_pk,enroll_pct_special_ed_pk,enroll_count_vocation_9_12,enroll_pct_vocation_9_12,enroll_count_alt_programs,enroll_pct_alt_programs,enroll_count_gifted,enroll_pct_gifted
count,15913.000000,15913.000000,15913.000000,15913.000000,15913.000000,15913.00000,15917.000000,15906.000000,15911.000000,15917.000000,...,15907.000000,15907.000000,15911.000000,15911.000000,3371.000000,3371.000000,15907.000000,15907.000000,14759.000000,14759.000000
mean,2.869352,0.102620,39.519827,12.573996,3.247659,41.52366,0.280769,65.990695,11.715794,7.550983,...,76.149997,5.123870,8.677456,15.799491,1142.911896,53.912133,28.362419,2.253209,182.952165,10.349617
std,5.804786,0.344224,30.875535,14.821417,1.991355,29.38793,1.181299,26.916958,7.329275,12.309330,...,1827.502704,9.541068,208.135653,31.775093,12524.965551,23.400025,680.535493,11.604956,4189.144830,8.434835
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.100000
25%,0.000000,0.000000,13.000000,4.000000,2.000000,11.00000,0.000000,47.000000,8.800000,1.000000,...,1.000000,0.200000,0.000000,0.000000,231.000000,40.950000,0.000000,0.000000,25.000000,4.600000
50%,1.000000,0.000000,32.000000,8.000000,3.000000,44.00000,0.000000,69.000000,11.000000,3.000000,...,11.000000,1.600000,0.000000,0.000000,564.000000,58.900000,0.000000,0.000000,54.000000,8.500000
75%,3.000000,0.000000,64.000000,15.000000,4.000000,67.00000,0.000000,90.000000,13.400000,8.000000,...,38.000000,5.400000,6.000000,12.800000,859.000000,70.450000,5.000000,0.700000,108.000000,13.600000
max,71.000000,12.000000,100.000000,98.000000,20.000000,99.00000,24.000000,100.000000,100.000000,100.000000,...,102993.000000,100.000000,10731.000000,759.400000,292654.000000,100.000000,33085.000000,100.000000,197970.000000,95.500000


In [81]:
# checking where there are holes in our data
esl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15923 entries, 0 to 2488
Data columns (total 35 columns):
detail_lvl_desc                  15923 non-null object
instn_number                     15923 non-null object
school_dstrct_cd                 15923 non-null object
long_school_year                 15923 non-null object
instn_name                       15923 non-null object
school_dstrct_nm                 15923 non-null object
grades_served_desc               15923 non-null object
enroll_percent_asian             15913 non-null float64
enroll_percent_native            15913 non-null float64
enroll_percent_black             15913 non-null float64
enroll_percent_hispanic          15913 non-null float64
enroll_percent_multiracial       15913 non-null float64
enroll_percent_white             15913 non-null float64
enroll_percent_migrant           15917 non-null float64
enroll_percent_ed                15906 non-null float64
enroll_percent_swd               15911 non-null float64
enr

It seems that most of our data is complete, with data fields null where they should be null (i.e., a statistic for elementary schools should be null for a high school)

In [82]:
esl_df = esl_df[['instn_number',
                 'school_dstrct_cd',
                 'long_school_year',
                 'instn_name',
                 'school_dstrct_nm',
                 'grades_served_desc',
                 'enroll_count_esol',
                 'enroll_pct_esol']]

In [84]:
esl_df.head()

,instn_number,school_dstrct_cd,long_school_year,instn_name,school_dstrct_nm,grades_served_desc,enroll_count_esol,enroll_pct_esol
0,0103,601,2011-12,Appling County High School,Appling County,"09,10,11,12",8.0,0.8
1,0109,601,2011-12,Baxley Wilderness Institute,Appling County,"07,08,09,10",0.0,0.0
2,0177,601,2011-12,Appling County Elementary School,Appling County,"03,04,05",9.0,1.7
3,0195,601,2011-12,Appling County Middle School,Appling County,"06,07,08",15.0,2.0
4,0277,601,2011-12,Appling County Primary School,Appling County,"PK,KK,01,02",82.0,13.6


# To be continued...